In [6]:
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [7]:

# Fetch DAX data
ticker_symbol = "^GDAXI"
dax_data = yf.download(ticker_symbol, start="1900-01-01", end="2025-01-13")

# Convert to a Pandas DataFrame
dax_df = pd.DataFrame(dax_data)

# Bundestagswahl data
elections_data = [
    ("1949-08-14", "CDU/CSU, FDP, DP"),
    ("1953-09-06", "CDU/CSU, FDP, DP, GB/BHE"),
    ("1957-09-15", "CDU/CSU, DP"),
    ("1961-09-17", "CDU/CSU, FDP"),
    ("1965-09-19", "CDU/CSU, SPD"),
    ("1969-09-28", "SPD, FDP"),
    ("1972-11-19", "SPD, FDP"),
    ("1976-10-03", "SPD, FDP"),
    ("1980-10-05", "SPD, FDP"),
    ("1983-03-06", "CDU/CSU, FDP"),
    ("1987-01-25", "CDU/CSU, FDP"),
    ("1990-12-02", "CDU/CSU, FDP"),
    ("1994-10-16", "CDU/CSU, FDP"),
    ("1998-09-27", "SPD, Greens"),
    ("2002-09-22", "SPD, Greens"),
    ("2005-09-18", "CDU/CSU, SPD"),
    ("2009-09-27", "CDU/CSU, FDP"),
    ("2013-09-22", "CDU/CSU, SPD"),
    ("2017-09-24", "CDU/CSU, SPD"),
    ("2021-09-26", "SPD, Greens, FDP")
]

[*********************100%***********************]  1 of 1 completed


In [19]:
def extract_prices(date):
    election_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = election_date - timedelta(weeks=6)
    end_date = election_date + timedelta(weeks=6)
    
    # Ensure we have data in the specified range
    if start_date < dax_df.index[0]:
        return None, None, None
    
    # Filter the DAX data for the relevant period
    period_data = dax_df[(dax_df.index >= start_date) & (dax_df.index <= end_date)]
    
    if not period_data.empty:
        start_price = period_data['Close'].iloc[0]
        
        # Find the closest date to the election date within the period
        nearest_date = period_data.index[np.abs(period_data.index - election_date).argmin()]
        mid_price = period_data['Close'].loc[nearest_date]
        
        end_price = period_data['Close'].iloc[-1]
        
        return start_price, mid_price, end_price
    
    return None, None, None

# Extract prices for each election and store results
prices = []
for date, parties in elections_data:
    start_price, mid_price, end_price = extract_prices(date)
    
    prices.append({
        'Election Date': date,
        'Governing Parties': parties,
        'Start Price': start_price,
        'Mid Price': mid_price,
        'End Price': end_price
    })

# Create DataFrame with election results and DAX prices
election_prices_df = pd.DataFrame(prices)

print(election_prices_df.tail(10))


   Election Date Governing Parties  \
10    1987-01-25      CDU/CSU, FDP   
11    1990-12-02      CDU/CSU, FDP   
12    1994-10-16      CDU/CSU, FDP   
13    1998-09-27       SPD, Greens   
14    2002-09-22       SPD, Greens   
15    2005-09-18      CDU/CSU, SPD   
16    2009-09-27      CDU/CSU, FDP   
17    2013-09-22      CDU/CSU, SPD   
18    2017-09-24      CDU/CSU, SPD   
19    2021-09-26  SPD, Greens, FDP   

                                          Start Price  \
10                                               None   
11  Ticker
^GDAXI    1474.160034
Name: 1990-10-22 ...   
12  Ticker
^GDAXI    2173.600098
Name: 1994-09-05 ...   
13  Ticker
^GDAXI    5432.029785
Name: 1998-08-17 ...   
14  Ticker
^GDAXI    3647.129883
Name: 2002-08-12 ...   
15  Ticker
^GDAXI    4837.859863
Name: 2005-08-08 ...   
16  Ticker
^GDAXI    5201.609863
Name: 2009-08-17 ...   
17  Ticker
^GDAXI    8359.25
Name: 2013-08-12 00:0...   
18  Ticker
^GDAXI    12165.120117
Name: 2017-08-14...   
19  Ticker


In [20]:
# Save the election performance data to a CSV file
election_prices_df.to_csv("data/election_dax_performance.csv", index=False)

In [20]:
# Calculate weekly returns and average weekly growth
dax_df['Weekly Return'] = dax_df['Close'].pct_change()  # Calculate weekly percentage change

# Group by week and calculate average weekly return
average_weekly_growth = dax_df['Weekly Return'].resample('W').mean() * 100  # Convert to percentage

# Calculate overall average weekly growth
overall_average_weekly_growth = average_weekly_growth.mean()

print(f"Average Weekly Growth of DAX: {overall_average_weekly_growth:.2f}%")

Average Weekly Growth of DAX: 0.05%


In [34]:
def calculate_performance(date):
    election_date = datetime.strptime(date, "%Y-%m-%d")
    start_date = election_date - timedelta(weeks=1)
    end_date = election_date + timedelta(weeks=12)
    
    # Ensure we have data in the specified range
    if start_date < dax_df.index[0]:
        return None, None
    
    # Filter the DAX data for the relevant period
    period_data = dax_df[(dax_df.index >= start_date) & (dax_df.index <= end_date)]
    
    if not period_data.empty:
        start_price = period_data['Close'].iloc[0]
        
        # Find the closest date to the election date within the period
        nearest_date = period_data.index[np.abs(period_data.index - election_date).argmin()]
        mid_price = period_data['Close'].loc[nearest_date]
        
        end_price = period_data['Close'].iloc[-1]
        
        return round(start_price, 2), round(end_price, 2)
    
    return None, None

# Calculate performance for each election and store results
performances = []
for date, parties in elections_data:
    before, after = calculate_performance(date)
    
    # Append results ensuring numerical values are stored directly
    performances.append({
        'Election Date': date,
        'Governing Parties': parties,
        'start_price Before (%)': before,
        'end_price After (%)': after
    })

# Create DataFrame with election results and DAX performance
election_performance_df = pd.DataFrame(performances)

# Clean up performance columns to ensure they are numeric and handle NaN values properly
election_performance_df['start_price Before (%)'] = election_performance_df['start_price Before (%)'].astype(float)
election_performance_df['end_price After (%)'] = election_performance_df['end_price After (%)'].astype(float)

print(election_performance_df.tail(10))

   Election Date Governing Parties  start_price Before (%)  \
10    1987-01-25      CDU/CSU, FDP                     NaN   
11    1990-12-02      CDU/CSU, FDP                 1458.28   
12    1994-10-16      CDU/CSU, FDP                 2048.56   
13    1998-09-27       SPD, Greens                 4439.13   
14    2002-09-22       SPD, Greens                 3319.05   
15    2005-09-18      CDU/CSU, SPD                 4989.98   
16    2009-09-27      CDU/CSU, FDP                 5668.65   
17    2013-09-22      CDU/CSU, SPD                 8613.00   
18    2017-09-24      CDU/CSU, SPD                12559.39   
19    2021-09-26  SPD, Greens, FDP                15132.06   

    end_price After (%)  
10                  NaN  
11              1574.77  
12              2058.20  
13              4666.74  
14              3077.06  
15              5282.13  
16              5831.21  
17              9006.46  
18             13103.56  
19             15531.69  


C:\Users\softw\AppData\Local\Temp\ipykernel_11932\3703343756.py:43: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['start_price Before (%)'] = election_performance_df['start_price Before (%)'].astype(float)
C:\Users\softw\AppData\Local\Temp\ipykernel_11932\3703343756.py:44: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  election_performance_df['end_price After (%)'] = election_performance_df['end_price After (%)'].astype(float)


In [35]:
# Save the election performance data to a CSV file
election_performance_df.to_csv("data/election_dax_performance.csv", index=False)

: 

In [4]:
import pandas as pd

# Create a DataFrame from the elections data
elections_df = pd.DataFrame(elections_data, columns=['Date', 'Coalition'])
elections_df['Date'] = pd.to_datetime(elections_df['Date'])

# Set the Date as the index of the DAX DataFrame
dax_df.index = pd.to_datetime(dax_df.index)

# Merge the DAX data with the elections data
result_df = pd.merge_asof(elections_df, dax_df['Close'], left_on='Date', right_index=True, direction='nearest')

# Rename the 'Close' column to 'DAX_Points'
result_df = result_df.rename(columns={'Close': 'DAX_Points'})

# Display the resulting DataFrame
print(result_df)


         Date                 Coalition        ^GDAXI
0  1949-08-14          CDU/CSU, FDP, DP   1005.190002
1  1953-09-06  CDU/CSU, FDP, DP, GB/BHE   1005.190002
2  1957-09-15               CDU/CSU, DP   1005.190002
3  1961-09-17              CDU/CSU, FDP   1005.190002
4  1965-09-19              CDU/CSU, SPD   1005.190002
5  1969-09-28                  SPD, FDP   1005.190002
6  1972-11-19                  SPD, FDP   1005.190002
7  1976-10-03                  SPD, FDP   1005.190002
8  1980-10-05                  SPD, FDP   1005.190002
9  1983-03-06              CDU/CSU, FDP   1005.190002
10 1987-01-25              CDU/CSU, FDP   1005.190002
11 1990-12-02              CDU/CSU, FDP   1466.400024
12 1994-10-16              CDU/CSU, FDP   2095.580078
13 1998-09-27               SPD, Greens   4677.560059
14 2002-09-22               SPD, Greens   2914.250000
15 2005-09-18              CDU/CSU, SPD   4926.129883
16 2009-09-27              CDU/CSU, FDP   5736.310059
17 2013-09-22              C

In [5]:
# Save the election performance data to a CSV file
result_df.to_csv("data/result_df.csv", index=False)